Вы сдаете в аренду помещения и получили предложение арендовать площади в количестве 780 м. кв. по установленной вами на основе рыночных данных цене 1500 руб. за 1 м. кв. в месяц, сроком на 1 год.

Вам предлагают следующую схему оплаты:

* разовый платеж в сумме 840 000 руб., затем,
* в конце 3-го, 4-го и 5-го месяцев платежи в размере 1.2 млн руб.,
* в начале 7-го, 8-го и 10-го месяцев – 1.2 млн руб. и
* в конце 12-го месяца – внесение последнего платежа в сумме 6 млн руб.

Выгодно ли данное предложение, если по стандартным условиям оплаты является ежемесячное внесение платежа равными долями? Ставка рефинансирования ЦБ равна 4.5% годовых.

## Задача 1

In [ ]:
import numpy as np
sq_m = 780
monthly_per_sq_m = 1500
months = 12
rate = 4.5

def standart_scheme(sq_m, monthly_per_sq_m, months, rate):
  PV = 0
  for k in range(1, months + 1):
    PV += monthly_per_sq_m * sq_m / ((1 + rate/100) ** (k/12))
  return PV

def custom_scheme(rate):
  PV = 840000
  pay_months = [3, 4, 5, 7, 8, 10]
  for k in pay_months:
    PV +=  1200000 / ((1 + rate/100) ** (k/12))
  PV += 6000000 / (1 + rate / 100) 
  return PV

print('standard scheme:', round(standart_scheme(sq_m, monthly_per_sq_m, months, rate), 2))
print('custom scheme:', round(custom_scheme(rate), 2))
print('difference = ', round(standart_scheme(sq_m, monthly_per_sq_m, months, rate) - custom_scheme(rate), 2))


standard scheme: 13710309.74
custom scheme: 13620867.2
difference =  89442.55


Ответ: нет, данное предложение невыгодно, стандартная схема выгоднее на 89 442.55 руб.

## Задача 2

Есть 2 возможности для вложения средств:
1. Депозит по ставке   4.56% на срок до 29.01.2023 с ежедневной капитализацией процентов
2. ОФЗ 26211 https://smart-lab.ru/q/bonds/SU26211RMFS1/ , доходность к погашению 4.68% годовых.

Предположим, что 30.09.2020 вы выбираете купить на  100 000 рублей облигаций или положить ту же сумму на депозит?

1. Сколько облигаций вы сможете купить на эти деньги?
2. Посчитайте чистую и грязную цену облигации.
3. Какой инструмент инвестирования вы выберете и почему?


In [ ]:
import datetime as dt
import pandas as pd
import numpy as np

start_date = '30-09-2020' 
end_date = '25-01-2023'
delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
rate = 4.56
sum = 100000

def daily_deposit(delta, rate, sum):
  print("number of days:", delta.days) 
  FV = sum * ((1 + rate /100 / 365) ** delta.days)
  return FV

print("deposit:", round(daily_deposit(delta, rate, sum) ,  2), "- to 25.01.2023, bond maturity date")   
#day count convention =  ACT / 365
face_value = 1000
coupon_rate = 3.5 # paid semiannually
dates = ['27-01-2021', '28-07-2021', '26-01-2022', '27-07-2022', '25-01-2023']
for i in range(len(dates) - 1): 
    coupon_delta = pd.to_datetime(dates[i + 1]) - pd.to_datetime(dates[i])
    #print(coupon_delta.days) #182

#coupons are paid in 182 days - actual
act = 182 
YTM = 4.68
def bond (act, dates, sum, face_value, coupon_rate, start_date, YTM):
  price_dirty = 0
  coupons = coupon_rate * face_value / 100 #coupons paid 
  days_from_latest_coupon = pd.to_datetime(dates[0]) - pd.to_datetime(start_date)
  accrued_interest = coupons * (act - days_from_latest_coupon.days) / 365 
  print("coupons",coupons, "NKD", accrued_interest, (act - days_from_latest_coupon.days),coupons)
  
  #clean_price 
  for i in range(len(dates)):
    price_dirty = price_dirty + coupons / pow(1 + YTM/100, ((pd.to_datetime(dates[i]) - pd.to_datetime(start_date)).days)/365)
    #print (price_dirty)
  
  price_dirty = price_dirty + face_value / pow(1 + YTM/100, ((pd.to_datetime(dates[len(dates)-1]) - pd.to_datetime(start_date)).days)/365)
  #print(price_dirty, face_value, pow(1 + YTM/100, ((pd.to_datetime(dates[len(dates)-1]) - pd.to_datetime(start_date)).days)/365))
  price_clean = price_dirty - accrued_interest
  number_of_bonds = sum / price_dirty
  

  print("number of bonds:", np.floor(number_of_bonds))
  print("clean price:",  round(price_clean, 2),  "dirty_price:", round(price_dirty, 2))
  FV = number_of_bonds * (coupons * len(dates) + face_value)
  return FV
  
print("bond:", round(bond(act, dates, sum, face_value, coupon_rate, start_date, YTM), 2))  

number of days: 847
deposit: 111161.11 - to 25.01.2023, bond maturity date
coupons 35.0 NKD 6.041095890410959 63 35.0
number of bonds: 93.0
clean price: 1058.07 dirty_price: 1064.11
bond: 110420.96


Ответ:
1. Можно купить 94 облигации
2. Чистая цена = 1051.6 руб., грязная = 1057.64 руб.
3. Сравним доход в один момент времени: 25.01.23 - дата последней выплаты по облигации. К этому времени депозит с ежедневной капитализацией процентов будет равен 111161.11 руб., а сумма всех выплат по облигации 110420.96 руб. Разница незначительна ( $<$ 1000 руб.), поэтому выбирать надо из соображений что важнее: финальная сумма или равномерный cashflow (ведь с депозита нельзя снять часть денег). Я выберу облигацию из-за равномерности выплат. 